In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import pytorch_lightning
from nlp_datasets import get_dataloader, get_dataset

import torch
from torch import optim
from torch.optim.lr_scheduler import StepLR
from tensorboardX import SummaryWriter
from accelerate import Accelerator
from safetensors.torch import load_file

import json
import gc

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
torch.cuda.empty_cache()
gc.collect()

98

In [5]:
text_decoder_id = f"upstage/SOLAR-10.7B-Instruct-v1.0"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(
    text_decoder_id,
    lagacy = False
)

tokenizer.pad_token = tokenizer.eos_token

In [6]:
base_model = AutoModelForCausalLM.from_pretrained(
    text_decoder_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True, 
    # quantization_config=bnb_config
    )

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [6]:
for name, module in base_model.named_modules():
    print(name)
    break

In [4]:
def save_to_json_rouge(output_filename, valid_idx, r1, r2, rl, r1_recall, r2_recall, rl_recall):
    metrics_data = {
        "valid_idx" : valid_idx,
        "rouge1": r1,
        "rouge2": r2,
        "rougel": rl,
        "rouge1_recall": r1_recall,
        "rouge2_recall": r2_recall,
        "rougel_recall" : rl_recall
    }
    with open(output_filename, "a+") as f:
        json.dump(metrics_data, f)
        f.write('\n')


In [5]:
from rouge_score import rouge_scorer
def get_rouge(tgt, hyp):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(tgt, hyp)
    r1 = scores["rouge1"].fmeasure
    recall_1 = scores["rouge1"].recall
    r2 = scores["rouge2"].fmeasure
    recall_2 = scores["rouge2"].recall
    rl = scores["rougeL"].fmeasure
    recall_l = scores["rougeL"].recall
    return r1, r2, rl, recall_1, recall_2, recall_l

In [6]:
ds = get_dataset(split="valid", get_prompt_only=True)
device = "cuda"

base_model.to(device)

for i in range(len(ds)) :
    inputs = tokenizer(ds[i]['prompt'], return_tensors="pt")

    with torch.no_grad():
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = base_model.generate(input_ids=inputs["input_ids"], max_new_tokens=512)
        generated_text = str(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))
    r1, r2, rl, recall_1, recall_2, recall_l = get_rouge(ds[i]['label'], generated_text)
    
    save_to_json_rouge("./baseline_eval_result_rouge.json", i, r1, r2, rl, recall_1, recall_2, recall_l)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:144783 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:144783 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:144783 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:144783 for open-end gene